# LOAD DATASET

In [1]:
import pandas as pd
import numpy as np


concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()
 


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# PREPROCESSING

check for null values

In [2]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [3]:

concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column
predictors.head()
 

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [4]:

target.head()
 


0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

normalising the tarining data 

In [5]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [6]:
 
n_cols = predictors.shape[1] # number of predictors
n_cols




8

# MODEL BUILDING

In [7]:

import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

# build the model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
 


# TRAINING

In [8]:
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/50
23/23 [==============================] - 1s 2ms/step - loss: 6128.2554
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 3085.3442
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 1390.2056
Epoch 4/50
23/23 [==============================] - 0s 1ms/step - loss: 666.3726
Epoch 5/50
23/23 [==============================] - 0s 1ms/step - loss: 533.5297
Epoch 6/50
23/23 [==============================] - 0s 1ms/step - loss: 459.6014
Epoch 7/50
23/23 [==============================] - 0s 1ms/step - loss: 403.7722
Epoch 8/50
23/23 [==============================] - 0s 1ms/step - loss: 358.9265
Epoch 9/50
23/23 [==============================] - 0s 1ms/step - loss: 328.4892
Epoch 10/50
23/23 [==============================] - 0s 1ms/step - loss: 301.1226
Epoch 11/50
23/23 [==============================] - 0s 1ms/step - loss: 278.9613
Epoch 12/50
23/23 [==============================] - 0s 1ms/step - loss: 260.2862
Epoch 13/50
23/23 [===

In [9]:
from sklearn.metrics import mean_squared_error
y_pred=model.predict(X_test)
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)
 


10/10 [==============================] - 0s 889us/step
138.10533019935994 0.0


# EVALUATION

In [11]:

total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with  normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 45.345489501953125
10/10 [==============================] - 0s 667us/step
MSE 2: 52.488990783691406
10/10 [==============================] - 0s 2ms/step
MSE 3: 45.82562255859375
10/10 [==============================] - 0s 956us/step
MSE 4: 47.379512786865234
10/10 [==============================] - 0s 1ms/step
MSE 5: 47.90459442138672
10/10 [==============================] - 0s 918us/step
MSE 6: 56.94124221801758
10/10 [==============================] - 0s 1ms/step
MSE 7: 55.55364990234375
10/10 [==============================] - 0s 2ms/step
MSE 8: 44.28974533081055
10/10 [==============================] - 0s 2ms/step
MSE 9: 47.901851654052734
10/10 [==============================] - 0s 3ms/step
MSE 10: 47.835845947265625
10/10 [==============================] - 0s 4ms/step
MSE 11: 45.570640563964844
10/10 [==============================] - 0s 3ms/step
MSE 12: 43.76219940185547
10/10 [==============================] - 0s 4ms/step
MSE 13: 56.89473342895508
10/10 [================

as we can see above the mean and standard deviation of MSE decreased as compared to the non normalised data